In [1]:
import requests,json,random,time,hashlib,math

def genSign(data):
    k1 = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9, "A": 10, "B": 11, "C": 12,
        "D": 13, "E": 14, "F": 15}
    l1 = [212, 45, 80, 68, 195, 163, 163, 203, 157, 220, 254, 91, 204, 79, 104, 6]
    t = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/="

    text = json.dumps(data, separators=(',', ':'))
    md5 = hashlib.md5(text.encode()).hexdigest().upper()

    t1 = ''.join([md5[i] for i in [21, 4, 9, 26, 16, 20, 27, 30]])
    t3 = ''.join([md5[i] for i in [18, 11, 3, 2, 1, 7, 6, 25]])

    ls2 = []
    for i in range(16):
        x1 = k1[md5[i * 2]]
        x2 = k1[md5[i * 2 + 1]]
        x3 = ((x1 * 16) ^ x2) ^ l1[i]
        ls2.append(x3)

    ls3 = []
    for i in range(6):
        if i == 5:
            ls3.append(t[ls2[-1] >> 2])
            ls3.append(t[(ls2[-1] & 3) << 4])
        else:
            x4 = ls2[i * 3] >> 2
            x5 = (ls2[i * 3 + 1] >> 4) ^ ((ls2[i * 3] & 3) << 4)
            x6 = (ls2[i * 3 + 2] >> 6) ^ ((ls2[i * 3 + 1] & 15) << 2)
            x7 = 63 & ls2[i * 3 + 2]
            ls3.extend(t[x4] + t[x5] + t[x6] + t[x7])

    t2 = ''.join(ls3).replace('[\\/+]', '')
    sign = 'zzb' + t1 + t2 + t3
    return sign.lower()

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
def fetchPlaylist(id):
    url = 'http://c.y.qq.com/qzone/fcg-bin/fcg_ucc_getcdinfo_byids_cp.fcg'
    data = {"type":1,"json":1,"utf8":1,"disstid":id,"loginUin":0}
    headers = {
        'referer': 'https://y.qq.com/n/yqq/playlist',
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
    }
    doc = requests.get(url,params=data,headers=headers).content
    l = json.loads(doc.decode('utf8').replace('jsonCallback(','')[:-1])
    mid_list = [(t['songmid'],t['songname'][:30]) for t in l['cdlist'][0]['songlist']]
    return mid_list

def fetchAlbumList(mid):
    url = 'https://u.y.qq.com/cgi-bin/musicu.fcg'
    data = {"albumSonglist":{"method":"GetAlbumSongList","module":"music.musichallAlbum.AlbumSongList","param":{"albumMid":mid,"albumID":0,"begin":0,"num":999,"order":2}}}
    params = {'data': json.dumps(data, separators=(',', ':'))}
    doc = json.loads(requests.get(url,params=params).content)
    if doc['code'] == 0:
        album_list = [(s['songInfo']['mid'], s['songInfo']['name']) for s in doc['albumSonglist']['data']['songList']]
    return album_list
    
def fetchFilename(mid,media_type=''):
    
    typeMap = {
    "m4a": ['C400','.m4a','size_96aac'],
    "128": ['M500','.mp3','size_128mp3'],
    "320": ['M800','.mp3','size_320mp3'],
    "ape": ['A000','.ape','size_ape'],
    "flac": ['F000','.flac','size_flac']
    }

    url = 'http://u.y.qq.com/cgi-bin/musicu.fcg'
    data = {"songinfo": {"method": "get_song_detail_yqq","module": "music.pf_song_detail_svr","param": {"song_mid": mid}}}
    params = {'data': json.dumps(data, separators=(',', ':'))}
    doc = requests.get(url,params=params).content
    info = json.loads(doc)

    if info['code'] == 0:
        file = info['songinfo']['data']['track_info']['file']
        media_mid  = file['media_mid']

        if media_type == '' or file[typeMap[media_type][2]] == 0:
            for t in typeMap.keys():
                if file[typeMap[t][2]] > 0:
                    media_type = t

        filename = typeMap[media_type][0]+media_mid+typeMap[media_type][1]
    else:
        filename = ''

    return filename


def fetchLink(mid,qqmusic_key):

    if 'Q' == qqmusic_key[0]:
        uin = '1451372807'
    else:
        uin = "1152921504615918390"
    url = 'https://u.y.qq.com/cgi-bin/musicu.fcg'
    data = {"comm":{"cv":4747474,"ct":24,"format":"json","inCharset":"utf-8","outCharset":"utf-8","notice":0,"platform":"yqq.json","needNewCode":1,"uin":uin,"authst": qqmusic_key},"req_0":{"module":"vkey.GetVkeyServer","method":"CgiGetVkey","param":{"guid":"9778124750","songmid":[mid],"songtype":[0],"uin":uin,"loginflag":1,"platform":"20","filename":[fetchFilename(mid)]}}}

    headers = {
        "content-type": "application/x-www-form-urlencoded",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
        #"cookie": f'wxuin={uin}; qm_keyst={qm_keys};',
    }

    params = {
        '_': math.floor(time.time()*1000),
        'sign': genSign(data),
        'data': json.dumps(data, separators=(',', ':'))
    }
    l = json.loads(requests.get(url, params=params, headers=headers).content)

    if l['code'] == 0:
        link = 'https://dl.stream.qqmusic.qq.com/' + l['req_0']['data']['midurlinfo'][0]['purl']
    else:
        print('Cookie expired')
        link = ''
    return link

In [ ]:
fetchAlbumList()

In [5]:
qqmusic_key = 'Q_H_L_5yA8-l56IiVE8xw7NP5oGAL9l9fHe-8pqN0r7o5NMw30tHMZPAyy4vA'
fetchLink('001OPw5a3M3iSf',qqmusic_key)

'https://dl.stream.qqmusic.qq.com/M500002Hgk1u1mKY6V.mp3?guid=9778124750&vkey=0C7DB8555C849DB5A3E0743E5FEA75530B87FA4D388990E18ADEE9B80DF37E1A21895BD2ACA052B0C4ED82134CC4FA810724D2ADDD3A94F7&uin=1451372807&fromtag=120042'